In [16]:
import numpy as np
import tensorflow as tf
import data
import data_utils
import importlib as I

In [ ]:
I.reload(data)

In [25]:
tf.reset_default_graph()


# placeholders
#x_ = [ tf.placeholder(tf.int64, [None, ], name = 'x_{}'.format(t)) for t in range(seqlen) ]
x_ = tf.placeholder(tf.int64, [None, seqlen], name = 'x')
y_ = tf.placeholder(tf.int64, [None, ], name = 'y')

# one-hot
x_onehot = tf.one_hot(x_, num_classes, axis=-1)
#rnn_inputs = tf.unpack(x_onehot, axis=1) # unpack along time axis
# http://stackoverflow.com/questions/38728501/inputs-not-a-sequence-wth-rnns-and-tensorflow

# rnn cell
cell = tf.nn.rnn_cell.BasicRNNCell(state_size)
rnn_outputs, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=x_onehot, dtype=tf.float32)

rnn_outputs = tf.transpose(rnn_outputs, perm=[1,0,2])
# parameters for softmax layer
W = tf.get_variable('W', [state_size, num_classes])
b = tf.get_variable('b', [num_classes],
        initializer=tf.constant_initializer(0.0))

# output for each time step
logits = tf.matmul(rnn_outputs[-1], W) + b
#predictions = tf.nn.softmax(logit)

# requires unnormalized prob
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y_)
loss = tf.reduce_mean(losses)

# train op
train_op = tf.train.AdagradOptimizer(learning_rate).minimize(loss)


In [28]:
logits.get_shape()

TensorShape([Dimension(None), Dimension(20)])

In [13]:
X, Y = data.gen_data()
(trainX,trainY), (testX,testY), (validX,validY) = data_utils.split_dataset(X, Y)
trainset = data.rand_batch_gen(trainX, trainY, batch_size=32)
testset = data.rand_batch_gen(testX, testY, batch_size=16)

In [ ]:
a,b = trainset.__next__()
a.shape

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batchX, batchY = trainset.__next__()
    _, lossv = sess.run([train_op, loss], feed_dict = {
            x_ : batchX,
            y_ : batchY,
        })

In [ ]:
lossv

## Model

In [29]:
import dynamic_rnn
I.reload(dynamic_rnn)

<module 'dynamic_rnn' from '/home/suriya/_/tf/TF/RNN/toys/binseq/dynamic_rnn.py'>

In [30]:
'''
seqlen = 20
num_classes = 20
state_size = 10
learning_rate = 0.1
'''
seqlen = 20
num_classes = 20
state_size = 10
learning_rate = 0.1
net = dynamic_rnn.Seq2Vec(seqlen=seqlen, num_classes=num_classes,
                          state_size=state_size, epochs=10000,
                          learning_rate=learning_rate,
                          ckpt_path='ckpt/' )

<log> Building Graph...</log>


In [31]:
sess, step = net.train(trainset)


>> Average train loss : 1.4119086060523987

>> Average train loss : 0.7031516264081001

>> Average train loss : 0.4727827562987804

>> Average train loss : 0.3638667490184307

>> Average train loss : 0.3034854677319527

>> Interrupted by user at iteration #5501


In [33]:
batchX, batchY = testset.__next__()
op = net.predict(sess, batchX)

In [35]:
batchY, op

(array([ 9, 10, 11,  9,  9, 10, 10, 12,  8, 11, 10, 12, 10, 11,  8,  7]),
 array([ 9, 10, 11,  9,  9, 10, 10, 12,  8, 11, 10, 12, 10, 11,  8,  7]))